# Installs

In [ ]:
# %pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q


This may take a while

In [ ]:
# # Uninstall the specific packages
# !pip uninstall torchsummaryx -y
# !pip uninstall wandb -y
# !pip uninstall python-Levenshtein -y
# !pip uninstall wget -y

# # If ctcdecode was installed, uninstall it as well
# !pip uninstall ctcdecode -y

# # Remove the ctcdecode directory if it was cloned
# import shutil
# import os

# if os.path.isdir("ctcdecode"):
#     shutil.rmtree("ctcdecode")


In [ ]:
# !pip install torchsummaryx==1.1.0
# !pip install wandb --quiet
# !pip install python-Levenshtein -q
# !git clone --recursive https://github.com/parlance/ctcdecode.git
# !pip install wget -q
# %cd ctcdecode
# !pip install . -q
# !python setup.py install
# %cd ..

fatal: destination path 'ctcdecode' already exists and is not an empty directory.
/content/ctcdecode
  Preparing metadata (setup.py) ... done
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/extension.py:139: UserWarning: Unknown Extension options: 'package', 'with_cuda'
  warnings.warn(msg)
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWa

# Imports

In [ ]:
#!pip install torchaudio
# !pip install pytorch-nlp

In [ ]:

# !pip uninstall ctcdecode

In [99]:
!pip install torchinfo
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torchnlp.nn import LockedDropout
from torchaudio.transforms import FrequencyMasking, TimeMasking
from torchinfo import summary
import torchaudio
import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [ ]:
# !pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8 -q
# !mkdir /root/.kaggle

# with open("/root/.kaggle/kaggle.json", "w+") as f:
#     f.write('{"username":"skandv","key":"0cafa647bf22cf57326d60803b03911c"}') # TODO: Put your kaggle username & key here

# !chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [100]:
%cd /content

/content


In [ ]:
# !kaggle competitions download -c 11-785-hw3p2-f24

11-785-hw3p2-f24.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#  '''
#  This will take a couple minutes, but you should see at least the following:
#  11-785-f24-hw3p2  ctcdecode  hw3p2asr-f24.zip  sample_data
#  '''
#  !unzip -q 11-785-hw3p2-f24.zip
# #  !ls

replace 11785-f24-hw3p2/dev-clean/mfcc/1272-128104-0000.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: 11785-f24-hw3p2  11-785-hw3p2-f24.zip  checkpoint  ctcdecode  sample_data  wandb


# Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Dataset and Dataloader

In [101]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]
# PHONEMES = sorted(PHONEMES)
# LABELS = sorted(LABELS)



In [104]:
# Printing the PHONEMES list to inspect its content and ensure it contains all expected phonemes.
# This is a good way to verify that the data was loaded or defined correctly.
print(PHONEMES)

# Printing the length of the PHONEMES list to ensure the number of phonemes matches expectations.
# This step helps confirm that there are no missing or duplicate entries in the phoneme list.
print(len(PHONEMES))

# Printing the LABELS list to inspect its content and validate that it aligns with the expected label mappings.
# This can serve as a sanity check to ensure that labels correspond accurately to the phonemes.
print(LABELS)

# Printing the length of the LABELS list to ensure it has the correct number of entries, ideally matching the number of phonemes.
# If the length does not match, it may indicate an issue with the mapping or data preprocessing.
print(len(LABELS))


['', '[SIL]', 'NG', 'F', 'M', 'AE', 'R', 'UW', 'N', 'IY', 'AW', 'V', 'UH', 'OW', 'AA', 'ER', 'HH', 'Z', 'K', 'CH', 'W', 'EY', 'ZH', 'T', 'EH', 'Y', 'AH', 'B', 'P', 'TH', 'DH', 'AO', 'G', 'L', 'JH', 'OY', 'SH', 'D', 'AY', 'S', 'IH']
41
[' ', '-', 'G', 'f', 'm', '@', 'r', 'u', 'n', 'i', 'W', 'v', 'U', 'o', 'a', 'R', 'h', 'z', 'k', 'C', 'w', 'e', 'Z', 't', 'E', 'y', 'A', 'b', 'p', 'T', 'D', 'c', 'g', 'l', 'j', 'O', 'S', 'd', 'Y', 's', 'I']
41


In [105]:
import os
import numpy as np

# Define root directory for dataset
dataset_root = '/content/11785-f24-hw3p2'
train_partition = 'train-clean-100'

# Define directories for MFCC and transcript files
mfcc_directory = os.path.join(dataset_root, train_partition, "mfcc")
transcript_directory = os.path.join(dataset_root, train_partition, "transcript")

# List all files in the directories in sorted order
mfcc_file_list = sorted(os.listdir(mfcc_directory))
transcript_file_list = sorted(os.listdir(transcript_directory))

# Initialize a list to store MFCC lengths
mfcc_time_steps = []

# Iterate through the MFCC and transcript files
for index in range(len(mfcc_file_list)):
    # Construct full paths for the current MFCC and transcript files
    mfcc_file_path = os.path.join(mfcc_directory, mfcc_file_list[index])
    transcript_file_path = os.path.join(transcript_directory, transcript_file_list[index])

    # Load the MFCC data
    mfcc_data = np.load(mfcc_file_path)
    mfcc_time_steps.append(mfcc_data.shape[0])  # Store the number of time steps

    # Load the transcript data
    transcript_data = np.load(transcript_file_path)

    # Print shapes of the MFCC and transcript data
    print(f"MFCC shape: {mfcc_data.shape}")
    print(f"Transcript shape: {transcript_data.shape}")

    # Break after processing the first 4 files as a sample
    if index == 3:
        break

MFCC shape: (1404, 28)
Transcript shape: (147,)
MFCC shape: (1590, 28)
Transcript shape: (196,)
MFCC shape: (1390, 28)
Transcript shape: (179,)
MFCC shape: (1467, 28)
Transcript shape: (183,)


### Train Data

In [118]:
import os
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence

class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, root, partition="train-clean-100"):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = os.path.join(root, partition, "mfcc")
        self.transcript_dir = os.path.join(root, partition, "transcript")

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        self.PHONEMES = PHONEMES

        # TODO: WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)

        # TODO: HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS
        self.phoneme_to_index = {phoneme: idx for idx, phoneme in enumerate(PHONEMES)}
        self.index_to_phoneme = {idx: phoneme for idx, phoneme in enumerate(PHONEMES)}

        # TODO: CREATE AN ARRAY OF ALL FEATURES AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
        self.mfccs, self.transcripts = [], []
        for i in range(self.length):
            mfcc_path = os.path.join(self.mfcc_dir, self.mfcc_files[i])
            transcript_path = os.path.join(self.transcript_dir, self.transcript_files[i])

            mfcc = np.load(mfcc_path)
            mfcc = mfcc - np.mean(mfcc, axis=0, keepdims=True)
            mfcc = mfcc / np.std(mfcc, axis=0, keepdims=True)

            transcript = np.load(transcript_path)[1:-1]
            self.mfccs.append(mfcc)
            self.transcripts.append([self.phoneme_to_index[i] for i in transcript])

    def __len__(self):
        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''
        mfcc = self.mfccs[ind]
        transcript = self.transcripts[ind]
        return mfcc, transcript

    def collate_fn(self, batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lengths of features,
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = [item[0] for item in batch]
        # batch of output phonemes
        batch_transcript = [torch.tensor(item[1], dtype=torch.long) for item in batch]

        lengths_mfcc = [mfcc.shape[0] for mfcc in batch_mfcc]
        lengths_transcript = [len(transcript) for transcript in batch_transcript]

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(
            [torch.tensor(mfcc, dtype=torch.float32) for mfcc in batch_mfcc],
            batch_first=True,
            padding_value=0.0,
        )

        # Use a custom padding value (-1) for the transcripts
        batch_transcript_pad = pad_sequence(
            batch_transcript, batch_first=True, padding_value=-1
        )

        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return (
            batch_mfcc_pad,
            batch_transcript_pad,
            torch.tensor(lengths_mfcc),
            torch.tensor(lengths_transcript),
        )

### Test Data

In [129]:
# Test DataLoader
class TestAudioDataset(torch.utils.data.Dataset):
    def __init__(self, root, partition="test-clean"):
        '''
        Initializes the test dataset.

        Args:
            root (str): Root directory containing the dataset.
            subset (str): Specific partition of the dataset (e.g., "test-clean").
        '''

        # Directory containing MFCC feature files
        self.feature_dir = os.path.join(root, partition, "mfcc")
        self.feature_files = sorted(os.listdir(self.feature_dir))

        # Total number of samples in the dataset
        self.dataset_size = len(self.feature_files)

        # Preload and normalize all MFCC features
        self.normalized_features = []
        for idx in range(self.dataset_size):
            feature_path = os.path.join(self.feature_dir, self.feature_files[idx])
            mfcc_data = np.load(feature_path)

            # Normalize MFCC data
            mfcc_normalized = (mfcc_data - np.mean(mfcc_data, axis=0, keepdims=True)) / np.std(
                mfcc_data, axis=0, keepdims=True
            )
            self.normalized_features.append(mfcc_normalized)

    def __len__(self):
        ''' Returns the total number of samples in the dataset. '''
        return self.dataset_size

    def __getitem__(self, index):
        '''
        Returns the normalized MFCC coefficients for the given index.

        Args:
            index (int): Index of the desired sample.

        Returns:
            numpy array: Normalized MFCC features for the sample.
        '''
        return self.normalized_features[index]

    def collate_fn(self, batch):
        '''
        Collate function to process a batch of test data.

        Args:
            batch: List of MFCC features.

        Returns:
            tuple: Padded MFCC features and their original lengths.
        '''

        # Convert batch items to tensors and determine their original lengths
        feature_tensors = [torch.tensor(features, dtype=torch.float32) for features in batch]
        feature_lengths = [features.shape[0] for features in feature_tensors]

        # Pad feature tensors to create a uniform batch
        padded_features = pad_sequence(
            feature_tensors,
            batch_first=True,
            padding_value=0.0  # Silence padding
        )

        # Return padded features and their lengths
        return padded_features, torch.tensor(feature_lengths)


### Config - Hyperparameters

In [130]:
root = "/content/11785-f24-hw3p2"

# Feel free to add more items here
config = {
            "batch_size" : 21,
            "epochs"     : 35,
            "beam_width" :  4,
            "lr"         : 2e-3
}

# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation
transforms = [] # set of tranformations

### Data loaders

In [131]:
# get me RAMMM!!!!
import gc
gc.collect()

1067

In [132]:
# Initialize Dataset Objects
# data_root: Path to the dataset directory
# subset: Specific partition of the dataset (e.g., "train-clean-100", "dev-clean", "test-clean")

train_dataset = AudioDataset(root, partition="train-clean-100")  # Updated variable names
validation_dataset = AudioDataset(root, partition="dev-clean")  # Consistent naming for validation set
test_dataset = TestAudioDataset(root, partition="test-clean")   # Updated to match test dataset class

# Create DataLoader objects for each dataset
# Ensure to pass the collate function for custom batching

train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=config["batch_size"],  # Adjustable based on system resources
    shuffle=True,                     # Shuffle training data
    collate_fn=train_dataset.collate_fn  # Collate function for batching
)

validation_data_loader = torch.utils.data.DataLoader(
    validation_dataset,
    batch_size=config["batch_size"],  # Adjustable based on system resources
    shuffle=False,                    # No shuffling for validation
    collate_fn=validation_dataset.collate_fn  # Collate function for batching
)

test_data_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=config["batch_size"],  # Adjustable based on system resources
    shuffle=False,                    # No shuffling for test data
    collate_fn=test_dataset.collate_fn  # Collate function for batching
)

# Print dataset and loader information
print("Batch size: ", config['batch_size'])
print("Training samples: {}, Training batches: {}".format(len(train_dataset), len(train_data_loader)))
print("Validation samples: {}, Validation batches: {}".format(len(validation_dataset), len(validation_data_loader)))
print("Test samples: {}, Test batches: {}".format(len(test_dataset), len(test_data_loader)))


Batch size:  21
Training samples: 28539, Training batches: 1359
Validation samples: 2703, Validation batches: 129
Test samples: 2620, Test batches: 125


In [133]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([21, 1602, 28]) torch.Size([21, 168]) torch.Size([21]) torch.Size([21])


# SeNet for Embeddings


In [134]:
class SqueezeExcitation1D(torch.nn.Module):
    def __init__(self, input_channels, reduction=16):
        super(SqueezeExcitation1D, self).__init__()
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.dense1 = nn.Linear(input_channels, input_channels // reduction)
        self.activation = nn.ReLU()
        self.dense2 = nn.Linear(input_channels // reduction, input_channels)
        self.scaling = nn.Sigmoid()

    def forward(self, input_tensor):
        batch_size, channels, _ = input_tensor.size()
        squeeze = self.global_pool(input_tensor).view(batch_size, channels)
        excitation = self.dense1(squeeze)
        excitation = self.activation(excitation)
        excitation = self.dense2(excitation)
        excitation = self.scaling(excitation).view(batch_size, channels, 1)
        return input_tensor * excitation


class ResidualBlock1D(torch.nn.Module):
    def __init__(self, input_channels, channel_filters):
        super(ResidualBlock1D, self).__init__()

        filter1, filter2 = channel_filters

        self.conv_block1 = nn.Conv1d(input_channels, filter1, kernel_size=3, stride=1, padding=1)
        self.norm1 = nn.BatchNorm1d(filter1)

        self.conv_block2 = nn.Conv1d(filter1, filter2, kernel_size=3, stride=1, padding=1)
        self.norm2 = nn.BatchNorm1d(filter2)

        self.squeeze_excite = SqueezeExcitation1D(filter2)
        self.activation = nn.ReLU()

        self.adjust_channels = None
        if input_channels != filter2:
            self.adjust_channels = nn.Conv1d(input_channels, filter2, kernel_size=1, stride=1)

    def forward(self, input_tensor):
        shortcut = input_tensor

        output = self.conv_block1(input_tensor)
        output = self.norm1(output)
        output = self.activation(output)

        output = self.conv_block2(output)
        output = self.norm2(output)

        output = self.squeeze_excite(output)

        if self.adjust_channels is not None:
            shortcut = self.adjust_channels(shortcut)

        output += shortcut
        output = self.activation(output)

        return output


class ResNet34(torch.nn.Module):
    def __init__(self, input_dim, feature_dim):
        super(ResNet34, self).__init__()

        self.input_layer = nn.Conv1d(input_dim, 192, kernel_size=7, stride=1, padding=3)
        self.initial_norm = nn.BatchNorm1d(192)
        self.initial_activation = nn.ReLU()

        self.block1 = nn.Sequential(
            ResidualBlock1D(192, [192, 192]),
            ResidualBlock1D(192, [192, 192]),
            ResidualBlock1D(192, [192, 192])
        )

        self.block2 = nn.Sequential(
            ResidualBlock1D(192, [384, 384]),
            ResidualBlock1D(384, [384, 384]),
            ResidualBlock1D(384, [384, 384]),
            ResidualBlock1D(384, [384, 384])
        )

        self.block3 = nn.Sequential(
            ResidualBlock1D(384, [768, 768]),
            ResidualBlock1D(768, [768, 768]),
            ResidualBlock1D(768, [768, 768]),
            ResidualBlock1D(768, [768, 768]),
            ResidualBlock1D(768, [768, 768]),
            ResidualBlock1D(768, [768, 768])
        )

        self.block4 = nn.Sequential(
            ResidualBlock1D(768, [feature_dim, feature_dim]),
            ResidualBlock1D(feature_dim, [feature_dim, feature_dim]),
            ResidualBlock1D(feature_dim, [feature_dim, feature_dim])
        )

    def forward(self, input_tensor):
        output = self.input_layer(input_tensor)
        output = self.initial_norm(output)
        output = self.initial_activation(output)

        output = self.block1(output)
        output = self.block2(output)
        output = self.block3(output)
        output = self.block4(output)

        return output


class ResNet18(torch.nn.Module):
    def __init__(self, input_dim, feature_dim):
        super(ResNet18, self).__init__()

        self.input_layer = nn.Conv1d(input_dim, 128, kernel_size=7, stride=1, padding=3)
        self.initial_norm = nn.BatchNorm1d(128)
        self.initial_activation = nn.ReLU()

        self.block1 = nn.Sequential(
            ResidualBlock1D(128, [128, 128]),
            ResidualBlock1D(128, [128, 128])
        )

        self.block2 = nn.Sequential(
            ResidualBlock1D(128, [256, 256]),
            ResidualBlock1D(256, [256, 256])
        )

        self.block3 = nn.Sequential(
            ResidualBlock1D(256, [512, 512]),
            ResidualBlock1D(512, [512, 512])
        )

        self.block4 = nn.Sequential(
            ResidualBlock1D(512, [feature_dim, feature_dim]),
            ResidualBlock1D(feature_dim, [feature_dim, feature_dim])
        )

    def forward(self, input_tensor):
        output = self.input_layer(input_tensor)
        output = self.initial_norm(output)
        output = self.initial_activation(output)

        output = self.block1(output)
        output = self.block2(output)
        output = self.block3(output)
        output = self.block4(output)

        return output


### Pyramid Bi-LSTM (pBLSTM)

In [135]:
# Utils for network
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [139]:
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input?
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''

    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()
        self.blstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=1, bidirectional=True, batch_first=True)

    def forward(self, x_packed):
        x_packed, _ = self.blstm(x_packed)
        x, x_lens = pad_packed_sequence(x_packed, batch_first=True)
        x, x_lens = self.trunc_reshape(x, x_lens)
        x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)

        return x_packed

    def trunc_reshape(self, x, x_lens):
        # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
        # TODO: Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        # TODO: Reduce lengths by the same downsampling factor
        if x.size(1) % 2:
          x = x[:, :-1]


        # Reshape by concatenating every two time steps
        batch_size, seq_len, feature_size = x.size()

        x = x.view(batch_size, seq_len // 2, feature_size * 2)

        # Update sequence lengths by dividing by 2
        x_lens = x_lens // 2

        return x, x_lens


### Encoder

In [140]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, encoder_hidden_size, num_layers = 2):
        super(Encoder, self).__init__()


        self.embedding = nn.Sequential(
            PermuteBlock(),
            ResNet34(input_size, encoder_hidden_size),
            PermuteBlock()
        )

        # Sequential pBLSTM layers
        self.pBLSTMs = nn.Sequential(

            pBLSTM(input_size=encoder_hidden_size,
                   hidden_size=encoder_hidden_size),

            pBLSTM(input_size=encoder_hidden_size * 4,
                   hidden_size=encoder_hidden_size))

    def forward(self, x, x_lens):
        # Apply embedding layer
        # Where are x and x_lens coming from? The dataloader
        #TODO: Call the embedding layer
        # TODO: Pack Padded Sequence
        # TODO: Pass Sequence through the pyramidal Bi-LSTM layer
        # TODO: Pad Packed Sequence
        x = self.embedding(x)

        # Pack, pass through pBLSTM layers, unpack
        x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        out_x_packed = self.pBLSTMs(x_packed)
        encoder_outputs, encoder_lens = pad_packed_sequence(out_x_packed, batch_first=True)

        return encoder_outputs, encoder_lens

### Decoder

In [142]:
class Decoder(torch.nn.Module):
    """
    Decoder module for transforming encoded feature embeddings into
    class probabilities using an MLP followed by a softmax layer.
    """

    def __init__(self, embed_size, output_size=41):
        """
        Initializes the Decoder module.

        Args:
            embed_size (int): The size of the input embeddings from the encoder.
            output_size (int): The number of output classes. Default is 41.
        """
        super(Decoder, self).__init__()

        # Define the multi-layer perceptron (MLP) for decoding
        self.mlp = nn.Sequential(
            PermuteBlock(),  # Custom block to permute dimensions (if required by input shape)
            nn.BatchNorm1d(embed_size),  # Normalize input embeddings across the batch
            PermuteBlock(),  # Restore the original dimension order after normalization
            nn.Linear(embed_size, 384),  # First fully connected layer: embed_size -> 384
            nn.ReLU(),  # Activation function to introduce non-linearity
            nn.Linear(384, 128),  # Second fully connected layer: 384 -> 128
            nn.ReLU(),  # Another ReLU activation for non-linearity
            nn.Linear(128, output_size)  # Final layer: 128 -> output_size (number of classes)
        )

        # Apply log softmax activation to generate class probabilities
        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        """
        Forward pass of the decoder.

        Args:
            encoder_out (torch.Tensor): Output from the encoder,
                                        shape (batch_size, seq_len, embed_size).

        Returns:
            torch.Tensor: Log-softmax probabilities of shape
                          (batch_size, seq_len, output_size).
        """
        out = self.mlp(encoder_out)  # Pass encoder output through the MLP
        out = self.softmax(out)  # Apply log softmax for class probabilities
        return out

In [143]:
class ASRModel(torch.nn.Module):
    """
    Automatic Speech Recognition (ASR) model combining augmentations,
    an encoder, and a decoder to convert audio features into text predictions.
    """

    def __init__(self, input_size, embed_size=512, output_size=len(PHONEMES)):
        """
        Initializes the ASR model with augmentation, encoder, and decoder components.

        Args:
            input_size (int): The size of the input feature vector (e.g., number of frequency bins).
            embed_size (int): The embedding size for the encoder hidden states. Default is 512.
            output_size (int): The number of output classes (e.g., phonemes). Default is length of PHONEMES.
        """
        super().__init__()

        # Define augmentation layers for the input features (frequency and time masking)
        self.augmentations = nn.Sequential(
            PermuteBlock(),  # Permute the input tensor for proper masking
            FrequencyMasking(freq_mask_param=10),  # Apply frequency masking (randomly masks parts of the frequency axis)
            TimeMasking(time_mask_param=20),  # Apply time masking (randomly masks parts of the time axis)
            PermuteBlock()  # Restore the input tensor's original dimension order after augmentations
        )

        # Define the encoder, which processes the input features and generates embeddings
        # The encoder uses RNNs or similar architectures to process the sequence of input features.
        self.encoder = Encoder(input_size, encoder_hidden_size=embed_size, num_layers=2)

        # Define the decoder, which takes the encoded embeddings and generates output predictions
        # The decoder uses MLPs or similar architectures to predict phonemes or other output classes.
        self.decoder = Decoder(embed_size * 4, output_size)

    def forward(self, x, lengths_x):
        """
        Forward pass of the ASR model.

        Args:
            x (torch.Tensor): Input feature tensor (e.g., spectrogram) of shape
                              (batch_size, seq_len, input_size).
            lengths_x (torch.Tensor): Lengths of the input sequences, needed to handle variable sequence lengths.

        Returns:
            decoder_out (torch.Tensor): The output of the decoder, representing predicted phonemes.
            encoder_lens (torch.Tensor): The lengths of the sequences output by the encoder.
        """
        # Apply augmentations only during training to improve generalization
        if self.training:
            x = self.augmentations(x)

        # Pass the input through the encoder to get the embeddings and sequence lengths
        encoder_out, encoder_lens = self.encoder(x, lengths_x)

        # Pass the encoder output through the decoder to get predictions
        decoder_out = self.decoder(encoder_out)

        return decoder_out, encoder_lens


## Initialize ASR Network

In [ ]:
# print(x.shape, lx.shape)

torch.Size([21, 1726, 28]) torch.Size([21])


In [ ]:
# !pip install torchinfo


In [144]:
model = ASRModel(
    input_size=28,  # MFCC feature size
    embed_size=856,
    output_size=len(PHONEMES)
).to(device)
print(model)
summary(model, input_data=x.to(device), lengths_x=lx)

ASRModel(
  (augmentations): Sequential(
    (0): PermuteBlock()
    (1): FrequencyMasking()
    (2): TimeMasking()
    (3): PermuteBlock()
  )
  (encoder): Encoder(
    (embedding): Sequential(
      (0): PermuteBlock()
      (1): ResNet34(
        (input_layer): Conv1d(28, 192, kernel_size=(7,), stride=(1,), padding=(3,))
        (initial_norm): BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (initial_activation): ReLU()
        (block1): Sequential(
          (0): ResidualBlock1D(
            (conv_block1): Conv1d(192, 192, kernel_size=(3,), stride=(1,), padding=(1,))
            (norm1): BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv_block2): Conv1d(192, 192, kernel_size=(3,), stride=(1,), padding=(1,))
            (norm2): BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (squeeze_excite): SqueezeExcitation1D(
              (global_pool): Adaptive

Layer (type:depth-idx)                                       Output Shape              Param #
ASRModel                                                     [21, 400, 41]             --
├─Encoder: 1-1                                               [21, 400, 3424]           --
│    └─Sequential: 2-1                                       [21, 1602, 856]           --
│    │    └─PermuteBlock: 3-1                                [21, 28, 1602]            --
│    │    └─ResNet34: 3-2                                    [21, 856, 1602]           39,227,491
│    │    └─PermuteBlock: 3-3                                [21, 1602, 856]           --
│    └─Sequential: 2-2                                       [6776, 3424]              --
│    │    └─pBLSTM: 3-4                                      [13563, 3424]             11,737,472
│    │    └─pBLSTM: 3-5                                      [6776, 3424]              29,323,136
├─Decoder: 1-2                                               [21, 400, 

# Training Config
Initialize Loss Criterion, Optimizer, CTC Beam Decoder, Scheduler, Scaler (Mixed-Precision), etc.

In [145]:
# Import necessary libraries
import torch

# CTC Loss: Connectionist Temporal Classification (CTC) is used for sequence-to-sequence tasks like ASR.
# CTC Loss allows training on unaligned data by considering all possible alignments of the input and target sequences.
# Reference: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
criterion = torch.nn.CTCLoss(blank=0, zero_infinity=True, reduction='mean')
# Explanation:
# - blank=0: The blank token represents the 'no label' token in the CTC loss computation.
# - zero_infinity=True: Ensures that infinity values are replaced by zero, useful for sequences with different lengths.
# - reduction='mean': Specifies that the loss should be averaged over all elements (across batch and time steps).

# Optimizer: AdamW optimizer is used for weight updates during training.
# AdamW is an improved version of Adam that decouples weight decay from the optimization steps.
optimizer = torch.optim.AdamW(model.parameters(), lr=config['lr'], weight_decay=1e-3)
# Explanation:
# - model.parameters(): This specifies the parameters of the model that will be updated during training.
# - lr=config['lr']: Learning rate, taken from the configuration (config).
# - weight_decay=1e-3: L2 regularization applied to the weights to prevent overfitting.

# CTC Beam Decoder: This decoder is used to decode the output of the model (predicted phonemes) using CTC decoding.
# It decodes the predicted probabilities into the most likely sequence of phonemes, using beam search for more accurate results.
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(
    labels=PHONEMES,  # The list of possible phoneme labels.
    blank_id=0,  # The index of the blank token used in CTC loss.
    beam_width=config['beam_width'],  # Beam search width for decoding (higher values increase search space).
    num_processes=4,  # Number of parallel processes for beam search decoding to speed up computation.
    log_probs_input=True  # If true, the input to the decoder is assumed to be log-probabilities.
)
# Explanation:
# - The decoder uses beam search to find the most likely output sequences by maintaining the top-k candidates at each decoding step.
# - beam_width controls the number of candidates considered during each decoding step (larger values improve accuracy but increase computation).

# Learning Rate Scheduler: Cosine Annealing Warm Restarts adjusts the learning rate using a cosine annealing schedule.
# It periodically restarts the learning rate schedule to avoid getting stuck in sharp local minima and improves training convergence.
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2, eta_min=1e-5)
# Explanation:
# - optimizer: The optimizer whose learning rate will be adjusted.
# - T_0=5: The number of iterations before the first restart (controls how often the learning rate restarts).
# - T_mult=2: Multiplier for the number of iterations between restarts. After each restart, the number of iterations increases.
# - eta_min=1e-5: Minimum learning rate value during annealing.

# Mixed Precision Training: This is used to accelerate training by using half-precision floating-point numbers (float16).
# The GradScaler helps to scale the gradients during backpropagation to prevent underflow when using lower precision.
scaler = torch.cuda.amp.GradScaler()
# Explanation:
# - scaler: Scales the gradients in mixed-precision training. This helps prevent overflow or underflow issues with lower precision.
# - Mixed-precision training improves performance by using 16-bit floating point operations instead of the standard 32-bit.


# Decode Prediction

In [177]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):
    """
    Decode the model's output using a beam search decoder and map the decoded indices to phonemes.

    Args:
        output (torch.Tensor): The model's output, typically of shape (batch_size, seq_length, num_classes).
        output_lens (torch.Tensor): The lengths of the output sequences for each batch element.
        decoder: The decoder used to decode the CTC output (e.g., CTCBeamDecoder).
        PHONEME_MAP (list): A mapping of phoneme indices to phoneme strings (default: LABELS).

    Returns:
        pred_strings (list): A list of predicted strings (phoneme sequences) for each element in the batch.
    """

    # Use the decoder to decode the model output (e.g., using beam search).
    # beam_results contains the top-k sequences, where each sequence has probabilities for each token.
    # out_lens are the actual lengths of the decoded sequences for each batch element.
    beam_results, _, _, out_lens = decoder.decode(output, seq_lens=output_lens)

    # List to store the predicted strings for each element in the batch.
    pred_strings = []

    # Iterate over the batch to decode each output and map it to phonemes.
    for i in range(output_lens.shape[0]):
        # Extract the predicted indices for the current example (beam_results[i][0] is the top candidate).
        # Use out_lens[i][0] to slice the sequence to the actual length of the prediction.
        predict_indices = beam_results[i][0][:out_lens[i][0]]

        # Convert the indices to a string of phonemes using the PHONEME_MAP.
        predict_string = ''.join([PHONEME_MAP[idx] for idx in predict_indices])

        # Append the decoded string to the list of predictions.
        pred_strings.append(predict_string)

    return pred_strings


def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS):
    """
    Calculate the average Levenshtein distance between the predicted sequences and the ground truth labels.

    Args:
        output (torch.Tensor): The model's output predictions (same shape as in decode_prediction).
        label (torch.Tensor): The ground truth label sequences (batch_size, max_label_length).
        output_lens (torch.Tensor): The lengths of the output sequences for each batch element.
        label_lens (torch.Tensor): The lengths of the label sequences for each batch element.
        decoder: The decoder used to decode the CTC output.
        PHONEME_MAP (list): A mapping of phoneme indices to phoneme strings (default: LABELS).

    Returns:
        dist (float): The average Levenshtein distance over the batch.
    """

    # Initialize the total Levenshtein distance to 0.
    dist = 0

    # Get the batch size (number of examples in the batch).
    batch_size = label.shape[0]

    # Decode the predicted sequences using the decoder.
    pred_strings = decode_prediction(output, output_lens, decoder, PHONEME_MAP)

    # Iterate over each example in the batch to calculate the Levenshtein distance.
    for i in range(batch_size):
        # Extract the predicted and ground truth strings for the current example.
        pred_string = pred_strings[i]

        # Extract the label sequence for the current example and convert it to a string using PHONEME_MAP.
        label_indices = label[i][:label_lens[i]]
        label_string = ''.join([PHONEME_MAP[idx] for idx in label_indices])

        # Compute the Levenshtein distance between the predicted and label strings and accumulate it.
        dist += Levenshtein.distance(pred_string, label_string)

    # Compute the average Levenshtein distance over the batch.
    dist /= batch_size  # Averaging the distance for the entire batch.

    return dist
    # Uncommented `return dist` at the end of function, as it's necessary for returning the computed distance.


# Test Implementation

In [149]:
# #test code to check shapes
# torch.cuda.empty_cache()

# model.eval()
# for i, data in enumerate(val_loader, 0):
#     x, y, lx, ly = data
#     x, y = x.to(device), y.to(device)
#     h, lh = model(x, lx)
#     print(h.shape)
#     h_permuted = torch.permute(h, (1, 0, 2))
#     print(h_permuted.shape, y.shape)
#     loss = criterion(h_permuted, y, lh, ly)
#     print(loss)

#     print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

#     break

# WandB

You will need to fetch your api key from wandb.ai

In [178]:
import wandb
wandb.login(key="f9cf09dac50889564c1f8fb34fff472412382ff7")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [179]:
run = wandb.init(
    name = "Resnet-34-runn", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

# Train Functions

In [162]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

## Training Setup

In [163]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

# Checkpoint Creation

In [164]:
# This is for checkpointing, if you're doing it over multiple sessions
import os

checkpoint_dir = '/content/checkpoint'

if not os.path.exists(checkpoint_dir):

    os.makedirs(checkpoint_dir)
last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
epoch_model_path = "/content/checkpoint/last_model.pth"
best_model_path = "/content/checkpoint/best_model.pth"

In [165]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    curr_lr = float(optimizer.param_groups[0]['lr'])
    train_loss = train_model(model, train_loader, criterion, optimizer)
    valid_loss, valid_dist = validate_model(model, val_loader, decoder)

    scheduler.step()

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,
        'valid_dist': valid_dist,
        'valid_loss': valid_loss,
        'lr'        : curr_lr
    })

    save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    wandb.save(epoch_model_path)
    print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        wandb.save(best_model_path)
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch: 1/35


Train:   3%|▎         | 44/1359 [00:58<30:00,  1.37s/it, loss=0.6220, lr=0.000698]

KeyboardInterrupt: 

# Generate Predictions and Submit to Kaggle

In [ ]:
checkpoint = torch.load("/content/checkpoint/best_model.pth")

model.load_state_dict(checkpoint['model_state_dict'])


In [ ]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

TEST_BEAM_WIDTH = 5

test_decoder = CTCBeamDecoder(
    labels=LABELS,
    blank_id=0,
    beam_width=TEST_BEAM_WIDTH,
    num_processes=4,
    log_probs_input=True
)
results = []

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)

    prediction_strings = decode_prediction(h, lh, test_decoder)
    #TODO save the output in results array.

    results.extend(prediction_strings)
    del x, lx, h, lh
    torch.cuda.empty_cache()

In [ ]:
df = pd.DataFrame({"index": range(len(results)), "label": results})

# Save to the specified file path
df.to_csv("/content/submission.csv", index=False)

In [ ]:
!kaggle competitions submit -c hw3p2-785-f24 -f /content/submission.csv -m "I made it!"
